# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

Explanation of each input and where it comes from.

In [14]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [15]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_02_standard_spectral.pkl")

In [16]:
# CHANNEL_MAP_AND_ALL_TRIALS_DF = pd.read_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")
# MERGED_TRIALS_AND_VIDEO = pd.read_pickle("./proc/trial_SLEAP_and_metadata.pkl")

In [17]:
theta_band = (4, 12)
gamma_band = (30, 50)

In [18]:
FONTSIZE=20

In [19]:
FULL_LFP_TRACES_PKL = "{}_03_standard_spectral_bands.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [20]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [21]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_trace', 'MD_lfp_trace', 'LH_lfp_trace',
       'BLA_lfp_trace', 'vHPC_lfp_trace',
       'mPFC_power_all_frequencies_all_windows',
       'MD_power_all_frequencies_all_windows',
       'LH_power_all_frequencies_all_windows',
       'BLA_power_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'power_timestamps',
       'power_calculation_frequencies',
       'BLA_LH_coherence_all_frequencies_all_windows',
       'BLA_MD_coherence_all_frequencies_all_windows',
       'BLA_mPFC_coherence_all_frequencies_all_windows',
       'BLA_vHPC_coherence_all_frequencies_all_windows',
   

## Filtering by theta and gamma band

- Filtering theta/gamma for power

In [22]:
power_columns = [col for col in LFP_SPECTRAL_DF.columns if "power" in col and "calculation" not in col and "time" not in col]

In [23]:
power_columns

['mPFC_power_all_frequencies_all_windows',
 'MD_power_all_frequencies_all_windows',
 'LH_power_all_frequencies_all_windows',
 'BLA_power_all_frequencies_all_windows',
 'vHPC_power_all_frequencies_all_windows']

In [24]:
LFP_SPECTRAL_DF["power_calculation_frequencies"].apply(lambda x: (x >= theta_band[0]) & (x <= theta_band[1])).iloc[0]

array([False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [25]:
for col in power_columns:
    brain_region_name = col.split("_")[0]
    print(brain_region_name)
    theta_power_col = "{}_power_theta".format(brain_region_name)
    gamma_power_col = "{}_power_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_power_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["power_calculation_frequencies"] >= theta_band[0]) & (x["power_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_power_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["power_calculation_frequencies"] >= gamma_band[0]) & (x["power_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

mPFC
MD
LH
BLA
vHPC


In [26]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,mPFC_power_theta,mPFC_power_gamma,MD_power_theta,MD_power_gamma,LH_power_theta,LH_power_gamma,BLA_power_theta,BLA_power_gamma,vHPC_power_theta,vHPC_power_gamma
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[5772.0, 2512.0, 2622.0, 6868.0, inf, inf, inf...","[110.56, 141.8, 138.6, 108.5, 113.5, 97.0, 132...","[inf, 7268.0, 5428.0, 6660.0, 5932.0, 5420.0, ...","[142.1, 91.5, 148.9, 204.2, 109.8, 94.9, 131.8...","[2774.0, 2794.0, 2554.0, 2378.0, 2150.0, 2050....","[92.25, 70.6, 63.84, 77.2, 53.2, 56.06, 79.5, ...","[5336.0, 4132.0, 3812.0, 5010.0, 4936.0, 3244....","[326.0, 348.2, 333.8, 246.5, 246.5, 222.9, 252...","[inf, inf, inf, inf, inf, inf, inf, inf, inf, ...","[499.5, 580.0, 545.5, 633.5, 499.0, 361.2, 727..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[5772.0, 2512.0, 2622.0, 6868.0, inf, inf, inf...","[110.56, 141.8, 138.6, 108.5, 113.5, 97.0, 132...","[inf, 7268.0, 5428.0, 6660.0, 5932.0, 5420.0, ...","[142.1, 91.5, 148.9, 204.2, 109.8, 94.9, 131.8...","[2774.0, 2794.0, 2554.0, 2378.0, 2150.0, 2050....","[92.25, 70.6, 63.84, 77.2, 53.2, 56.06, 79.5, ...","[5336.0, 4132.0, 3812.0, 5010.0, 4936.0, 3244....","[326.0, 348.2, 333.8, 246.5, 246.5, 222.9, 252...","[inf, inf, inf, inf, inf, inf, inf, inf, inf, ...","[499.5, 580.0, 545.5, 633.5, 499.0, 361.2, 727..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[1991.0, 2208.0, 4588.0, 3286.0, 2872.0, 2076....","[233.1, 373.2, 328.8, 294.0, 307.0, 416.0, 394...","[425.0, 445.2, 744.5, 686.0, 477.8, 551.5, 456...","[23.12, 18.11, 14.81, 16.69, 15.0, 16.3, 13.81...","[465.8, 519.5, 809.0, 779.5, 537.0, 674.5, 574...","[27.88, 21.25, 19.97, 26.27, 21.88, 23.75, 20....","[2288.0, 3380.0, 5130.0, 3960.0, 3568.0, 4868....","[575.0, 567.0, 551.0, 519.0, 432.5, 424.8, 357...","[3144.0, 3464.0, 5550.0, 5344.0, 4468.0, 4612....","[374.8, 488.8, 293.5, 310.5, 282.8, 376.0, 362..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[1991.0, 2208.0, 4588.0, 3286.0, 2872.0, 2076....","[233.1, 373.2, 328.8, 294.0, 307.0, 416.0, 394...","[425.0, 445.2, 744.5, 686.0, 477.8, 551.5, 456...","[23.12, 18.11, 14.81, 16.69, 15.0, 16.3, 13.81...","[465.8, 519.5, 809.0, 779.5, 537.0, 674.5, 574...","[27.88, 21.25, 19.97, 26.27, 21.88, 23.75, 20....","[2288.0, 3380.0, 5130.0, 3960.0, 3568.0, 4868....","[575.0, 567.0, 551.0, 519.0, 432.5, 424.8, 357...","[3144.0, 3464.0, 5550.0, 5344.0, 4468.0, 4612....","[374.8, 488.8, 293.5, 310.5, 282.8, 376.0, 362..."
4,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405]

- Filtering theta/gamma for coherence

In [27]:
coherence_columns = [col for col in LFP_SPECTRAL_DF.columns if "coherence" in col and "calculation" not in col and "time" not in col]

In [28]:
coherence_columns

['BLA_LH_coherence_all_frequencies_all_windows',
 'BLA_MD_coherence_all_frequencies_all_windows',
 'BLA_mPFC_coherence_all_frequencies_all_windows',
 'BLA_vHPC_coherence_all_frequencies_all_windows',
 'LH_MD_coherence_all_frequencies_all_windows',
 'LH_mPFC_coherence_all_frequencies_all_windows',
 'LH_vHPC_coherence_all_frequencies_all_windows',
 'MD_mPFC_coherence_all_frequencies_all_windows',
 'MD_vHPC_coherence_all_frequencies_all_windows',
 'mPFC_vHPC_coherence_all_frequencies_all_windows']

In [29]:
for col in coherence_columns:
    brain_region_name = "_".join(col.split("_")[:2])
    print(brain_region_name)
    theta_coherence_col = "{}_coherence_theta".format(brain_region_name)
    gamma_coherence_col = "{}_coherence_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_coherence_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["coherence_calculation_frequencies"] >= theta_band[0]) & (x["coherence_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_coherence_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["coherence_calculation_frequencies"] >= gamma_band[0]) & (x["coherence_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

BLA_LH
BLA_MD
BLA_mPFC
BLA_vHPC
LH_MD
LH_mPFC
LH_vHPC
MD_mPFC
MD_vHPC
mPFC_vHPC


- Filtering theta/gamma for granger

In [30]:
granger_columns = [col for col in LFP_SPECTRAL_DF.columns if "granger" in col and "calculation" not in col and "time" not in col]

In [31]:
granger_columns

['BLA_LH_granger_all_frequencies_all_windows',
 'LH_BLA_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'MD_BLA_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'mPFC_BLA_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'vHPC_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'MD_LH_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'mPFC_LH_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'vHPC_LH_granger_all_frequencies_all_windows',
 'MD_mPFC_granger_all_frequencies_all_windows',
 'mPFC_MD_granger_all_frequencies_all_windows',
 'MD_vHPC_granger_all_frequencies_all_windows',
 'vHPC_MD_granger_all_frequencies_all_windows',
 'mPFC_vHPC_granger_all_frequencies_all_windows',
 'vHPC_mPFC_granger_all_frequencies_all_windows']

In [32]:
for col in granger_columns:
    brain_region_name = "-to-".join(col.split("_")[:2])
    print(brain_region_name)
    theta_granger_col = "{}_granger_theta".format(brain_region_name)
    gamma_granger_col = "{}_granger_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_granger_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["granger_calculation_frequencies"] >= theta_band[0]) & (x["granger_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_granger_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["granger_calculation_frequencies"] >= gamma_band[0]) & (x["granger_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

BLA-to-LH
LH-to-BLA
BLA-to-MD
MD-to-BLA
BLA-to-mPFC
mPFC-to-BLA
BLA-to-vHPC
vHPC-to-BLA
LH-to-MD
MD-to-LH
LH-to-mPFC
mPFC-to-LH
LH-to-vHPC
vHPC-to-LH
MD-to-mPFC
mPFC-to-MD
MD-to-vHPC
vHPC-to-MD
mPFC-to-vHPC
vHPC-to-mPFC


In [33]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,mPFC-to-MD_granger_theta,mPFC-to-MD_granger_gamma,MD-to-vHPC_granger_theta,MD-to-vHPC_granger_gamma,vHPC-to-MD_granger_theta,vHPC-to-MD_granger_gamma,mPFC-to-vHPC_granger_theta,mPFC-to-vHPC_granger_gamma,vHPC-to-mPFC_granger_theta,vHPC-to-mPFC_granger_gamma
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.6147, 0.2046, 0.413, 0.2115, 0.07513, 0.379...","[0.4363, 0.1776, 0.1467, 0.321, 0.1622, 0.1405...","[0.2495, 0.4895, 0.5493, 0.5884, 0.781, 0.7417...","[0.2996, 0.2195, 0.2698, 0.2234, 0.771, 0.303,...","[0.74, 0.83, 0.4905, 0.1929, 0.525, 0.2617, 0....","[0.3894, 0.2795, 0.3699, 0.3743, 0.364, 0.2167...","[0.2915, 0.325, 0.513, 0.11694, 0.1794, 0.356,...","[0.4688, 0.501, 0.5137, 0.257, 0.1731, 0.2177,...","[0.384, 0.3965, 0.4026, 0.07916, 0.3306, 0.237...","[0.263, 0.1315, 0.1389, 0.5635, 0.398, 0.3162,..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.6147, 0.2046, 0.413, 0.2115, 0.07513, 0.379...","[0.4363, 0.1776, 0.1467, 0.321, 0.1622, 0.1405...","[0.2495, 0.4895, 0.5493, 0.5884, 0.781, 0.7417...","[0.2996, 0.2195, 0.2698, 0.2234, 0.771, 0.303,...","[0.74, 0.83, 0.4905, 0.1929, 0.525, 0.2617, 0....","[0.3894, 0.2795, 0.3699, 0.3743, 0.364, 0.2167...","[0.2915, 0.325, 0.513, 0.11694, 0.1794, 0.356,...","[0.4688, 0.501, 0.5137, 0.257, 0.1731, 0.2177,...","[0.384, 0.3965, 0.4026, 0.07916, 0.3306, 0.237...","[0.263, 0.1315, 0.1389, 0.5635, 0.398, 0.3162,..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.4614, 0.2338, 0.3623, 0.04843, 0.1731, 0.18...","[0.3748, 0.2769, 0.2842, 0.3472, 0.4531, 0.200...","[0.10876, 0.1489, 0.1697, 0.5156, 0.584, 0.379...","[0.2188, 0.2308, 0.5767, 0.3372, 0.42, 0.287, ...","[0.812, 0.8457, 1.024, 0.7812, 0.8496, 0.3982,...","[0.2037, 0.199, 0.2454, 0.4004, 0.2162, 0.3481...","[0.329, 0.2783, 0.3787, 0.3135, 0.1807, 0.2905...","[0.343, 0.382, 0.6562, 0.3772, 0.4927, 0.4204,...","[0.3115, 0.3447, 1.015, 1.297, 0.367, 0.6484, ...","[0.3955, 0.2686, 0.2551, 0.3315, 0.238, 0.5864..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.4614, 0.2338, 0.3623, 0.04843, 0.1731, 0.18...","[0.3748, 0.2769, 0.2842, 0.3472, 0.4531, 0.200...","[0.10876, 0.1489, 0.1697, 0.5156, 0.584, 0.379...","[0.2188, 0.2308, 0.5767, 0.3372, 0.42, 0.287, ...","[0.812, 0.8457, 1.024, 0.7812, 0.8496, 0.3982,...","[0.2037, 0.199, 0.2454, 0.4004, 0.2162, 0.3481...","[0.329, 0.2783, 0.3787, 0.3135, 0.1807, 0.2905...","[0.343, 0.382, 0.6562, 0.3772, 0.4927, 0.4204,...","[0.3115, 0.3447, 1.015, 1.297, 0.367, 0.6484, ...","[0.3955, 0.2686, 0.2551, 0.3315, 0.238, 0.5864..."
4,2,20230612_112630_standard_com

In [38]:
for col in LFP_SPECTRAL_DF.columns:
    if "lfp_trace" in col and "region" not in col:
        brain_region = col.split("_")[0]
        LFP_SPECTRAL_DF["{}_lfp_zscore".format(brain_region)] = LFP_SPECTRAL_DF[col].apply(lambda x: stats.zscore(x.astype(int)).astype(np.float32))

In [39]:
LFP_SPECTRAL_DF

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,vHPC-to-MD_granger_gamma,mPFC-to-vHPC_granger_theta,mPFC-to-vHPC_granger_gamma,vHPC-to-mPFC_granger_theta,vHPC-to-mPFC_granger_gamma,mPFC_lfp_zscore,MD_lfp_zscore,LH_lfp_zscore,BLA_lfp_zscore,vHPC_lfp_zscore
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.3894, 0.2795, 0.3699, 0.3743, 0.364, 0.2167...","[0.2915, 0.325, 0.513, 0.11694, 0.1794, 0.356,...","[0.4688, 0.501, 0.5137, 0.257, 0.1731, 0.2177,...","[0.384, 0.3965, 0.4026, 0.07916, 0.3306, 0.237...","[0.263, 0.1315, 0.1389, 0.5635, 0.398, 0.3162,...","[0.1849693, 0.15769534, 0.1876967, 0.25666088,...","[0.12207468, 0.13095032, 0.20361966, 0.2679681...","[0.21378689, 0.22991511, 0.28531206, 0.3161660...","[0.15479894, 0.1536909, 0.20576839, 0.24233344...","[0.07216464, 0.11044584, 0.18987933, 0.2587855..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.3894, 0.2795, 0.3699, 0.3743, 0.364, 0.2167...","[0.2915, 0.325, 0.513, 0.11694, 0.1794, 0.356,...","[0.4688, 0.501, 0.5137, 0.257, 0.1731, 0.2177,...","[0.384, 0.3965, 0.4026, 0.07916, 0.3306, 0.237...","[0.263, 0.1315, 0.1389, 0.5635, 0.398, 0.3162,...","[0.1849693, 0.15769534, 0.1876967, 0.25666088,...","[0.12207468, 0.13095032, 0.20361966, 0.2679681...","[0.21378689, 0.22991511, 0.28531206, 0.3161660...","[0.15479894, 0.1536909, 0.20576839, 0.24233344...","[0.07216464, 0.11044584, 0.18987933, 0.2587855..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.2037, 0.199, 0.2454, 0.4004, 0.2162, 0.3481...","[0.329, 0.2783, 0.3787, 0.3135, 0.1807, 0.2905...","[0.343, 0.382, 0.6562, 0.3772, 0.4927, 0.4204,...","[0.3115, 0.3447, 1.015, 1.297, 0.367, 0.6484, ...","[0.3955, 0.2686, 0.2551, 0.3315, 0.238, 0.5864...","[0.2407101, 0.3648873, 0.41557187, 0.47639334,...","[0.68079615, 0.66690475, 0.59744775, 0.5326212...","[0.63482577, 0.56330836, 0.48732105, 0.5052004...","[0.7763766, 1.0399162, 1.0826523, 0.81199, 1.0...","[0.84525853, 1.1800785, 1.4270769, 1.182823, 1..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.2037, 0.199, 0.2454, 0.4004, 0.2162, 0.3481...","[0.329, 0.2783, 0.3787, 0.3135, 0.1807, 0.2905...","[0.343, 0.382, 0.6562, 0.3772, 0.4927, 0.4204,...","[0.3115, 0.3447, 1.015, 1.297, 0.367, 0.6484, ...","[0.3955, 0.2686, 0.2551, 0.3315, 0.238, 0.5864...","[0.2407101, 0.3648873, 0.41557187, 0.47639334,...","[0.68079615, 0.66690475, 0.59744775, 0.5326212...","[0.63482577, 0.56330836, 0.48732105, 0.5052004...","[0.7763766, 1.0399162, 1.0826523, 0.81199, 1.0...","[0.84525853, 1.1800785, 1.4270769, 1.182823, 1..."
4,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720]

In [40]:
LFP_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))